<a href="https://colab.research.google.com/github/satya-ip/ai/blob/main/Summarize_YouTube_Modern_AI_Pro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ModernAI Pro: From Youtube to Audio summaries

---





In this notebook let's use Whisper and ChatGPT to summarize a Youtube video

## 1.Downloading Youtube videos
Note: The legality of this depends on the geography

In [ ]:
!pip install -q -U pytubefix
from pytubefix import YouTube

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 730.7/730.7 kB 11.9 MB/s eta 0:00:00


In [ ]:
#url = "https://www.youtube.com/watch?v=zAjJYkUnTEs" #short
#url = "https://www.youtube.com/watch?v=suQYoqqomX8"
#url = "https://www.youtube.com/watch?v=VqspygBClOg"
#url = "https://www.youtube.com/watch?v=T-D1OfcDW1M"
url = "https://www.youtube.com/watch?v=3x4zM7O8pZI" #bb
#url = "https://www.youtube.com/watch?v=4Prc1UfuokY" #thisissparta
yt = YouTube(url)
audio = yt.streams.filter(only_audio=True).first().download()

BotDetection: 3x4zM7O8pZI This request was detected as a bot. Use `use_po_token=True` or switch to WEB client to view. See more details at https://github.com/JuanBindez/pytubefix/pull/209

In [ ]:
#Test if the Audio plays before we use whisper
from IPython.display import Audio
Audio(audio, rate = 22050)

## 2. Trancribe Audio with Whisper

In [ ]:
#First we will setup whisper
! pip install git+https://github.com/openai/whisper.git -q
import whisper
model = whisper.load_model("small") #for higher quality go for medium or large

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 107.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 80.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 90.9 MB/s et

100%|███████████████████████████████████████| 461M/461M [00:04<00:00, 97.1MiB/s]
/usr/local/lib/python3.11/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper

In [ ]:
transcript = model.transcribe(audio,fp16=False)['text']

In [ ]:
transcript

## 3. Clean the sentences and summarize with LLMs

In [ ]:
!pip -q install langchain-groq

In [ ]:
from google.colab import userdata
import os
os.environ["GROQ_API_KEY"] = userdata.get("GROQ_API_KEY")
from langchain_groq import ChatGroq
llm = ChatGroq(model_name="llama3-70b-8192")

In [ ]:
response = llm.invoke("Can you summarize and provide context to this audio.I'm new to LLMs: "+transcript).content
response

In [ ]:
def summarize_yt(url, llm):
  yt = YouTube(url)
  audio = yt.streams.filter(only_audio=True).first().download()
  print("Audio downloaded")

  transcript = model.transcribe(audio,fp16=False)['text']
  print(transcript)

  result = llm.invoke("Guess the movie and provide the context:"+transcript).content

  return result

In [ ]:
text = summarize_yt("https://www.youtube.com/watch?v=J-lDwmvpav0", llm)
text

In [ ]:
text=summarize_yt("https://www.youtube.com/watch?v=yhNWsX9PjeE",llm)
text

In [ ]:
text=summarize_yt("https://www.youtube.com/watch?v=omzW1NPIPvs",llm)
text